In [1]:
#!pip install scikit-surprise
%run -i utilities.py
%run -i CFKnnMeansModel_Class.py

# Model 1: Collab Filtering: KNN with means

## Read the model from saved file

In [2]:
import pickle
knn_filename = 'Model_KNN_Means.sav'
knn_model_full = pickle.load(open(knn_filename, 'rb'))
knn_model=knn_model_full.model

# Model 2: Content Based, Cosine Similarity

## Directly read Femke's notebook

> Important to identify the following:
> cosine_sim and cosine_sim_df

In [3]:
%run 2_ContentBasedFiltering.ipynb 

Shape: (9719, 9719)


# Hybrid Model
Hybrid Recommender leverages the best of both Content based and collaborative filtering techniques.

In [4]:
# Hybrid model
class HybridModel:
    def __init__(self, cosine_sim, cf_model):
        self.cosine_sim = cosine_sim
        self.cf_model = cf_model
        
    def recommend_movies(self, user_title_year, movies_df):
        
        # Use the Process_Avg_Rating function to manipulate the main df and find the 
        # avg rating
        
        movies_df_summary=Process_Avg_Rating(movies_df)
        
        #--------------------------------------
        # Content Based
        #--------------------------------------
        
        # Find the top 100 similar movies based on the content-based model
        similar_movies_cos_sim=genre_recommendation(user_title_year)
        
          
        #Merge
        similar_movies_cos_sim_df=pd.merge(similar_movies_cos_sim,movies_df_summary,how='left', left_on=['title','genres'], right_on = ['title','genres'])
        
        #--------------------------------------
        # Col. filter Based
        #--------------------------------------
        
        # Find the top 100 similar movies based on the Coll filter model
        similar_movies_knn=self.cf_model.recommend_similar_items_knnmeans(user_input,movies_df,100)
        
        #Take the common 10 movies
       
        similar_movies_common=pd.merge(similar_movies_knn,similar_movies_cos_sim_df, how='inner', on=['title','average rating','number of ratings'])
        similar_movies_common=similar_movies_common.sort_values(['average rating'], ascending=[False]).head(10)

        #What if most common movies are <10?
        
        return similar_movies_cos_sim_df, similar_movies_knn,similar_movies_common

## Read the df, and Ask for input

In [5]:
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
df=pd.read_csv("../../Data/ml-latest-small/PreprocessedData_ml_latest_year_small.csv")
#df["title"] = df["title"].str.lower()
df.head()

,userId,movieId,rating,title,genres,year
0,1,1,4.0,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1995
1,5,1,4.0,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1995
2,7,1,4.5,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1995
3,15,1,2.5,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1995
4,17,1,4.5,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1995


In [6]:
#Ask user for the movie name
user_input=AskForUserInput(df)
#Or if you want to hardcode it
#user_input="Toy Story"

Enter your Favorite Movie: Jumanji

We have multiple movies with the same name/Part of it, but with different release years:
 movieId                                 title  year
       2                        Jumanji (1995)  1995
  179401 Jumanji: Welcome to the Jungle (2017)  2017
Which one do you have in your mind? (Enter the movieId)2
Your favourite movie: Jumanji (1995)



## Recommend movies


In [7]:
hybrid_model=HybridModel(cosine_sim, knn_model)

In [8]:
r_content,r_cf,r_hybrid=hybrid_model.recommend_movies(user_input,df)
print("Recommendations, Content based:")
r_content.head(10)

AttributeError: 'KNNWithMeans' object has no attribute 'recommend_similar_items_knnmeans'

In [ ]:
print("Recommendations based on CF:")
r_cf.head(10)

In [ ]:
print("Recommendations, naive hybrid:")

r_hybrid.head(10)

# Save your model

In [ ]:
import pickle
filename = 'Model_hybrid.sav'
pickle.dump(hybrid_model, open(filename, 'wb'))